In [52]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

In [53]:
# load environment variables
load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "genai_langgraph_intro"

In [54]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [55]:
llm.invoke("hello")

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--49a60262-bf71-48a9-bbe3-8b32542a8da7-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

### Simple chatbot

In [7]:
while True:
    question = input("Ask a question: ")
    if question == "exit":
        break
    response = llm.invoke(question)
    print(response.content)

The Apple M4 chip is expected to be the next generation of Apple Silicon, likely powering future Mac Mini models with significant performance and efficiency improvements. While not officially announced, the M4 is anticipated to bring advancements in CPU, GPU, and Neural Engine capabilities, making the next Mac Mini a powerful and versatile compact computer.


### Chatbot with memory

In [58]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage


In [59]:
store = {}
config = {"configurable": {"session_id": "session_setup"}}

In [60]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [61]:
## update LLM with memory
llm_with_memory = RunnableWithMessageHistory(llm, get_session_history)

In [62]:
llm_with_memory.invoke("hello, I am subbu", config=config).content

"Hello Subbu! It's nice to meet you. How can I help you today?"

In [63]:
llm_with_memory.invoke("what is my name", config=config).content

'Your name is **Subbu**.'

In [64]:
store

{'session_setup': InMemoryChatMessageHistory(messages=[HumanMessage(content='hello, I am subbu', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Subbu! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--3258977d-2c7b-4833-9af6-b22b627060ab-0', usage_metadata={'input_tokens': 7, 'output_tokens': 19, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is **Subbu**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ebcfbdb9-049f-4999-87fc-a8979659efe3-0', usage_metadata={'input_tokens': 32, 'output_tokens': 7, 'total_tokens': 39, 'input_token_details': {'cache_read': 0}})])}

### RAG pipeline

In [65]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [66]:
## reading text file from directory

loader = DirectoryLoader("../data", glob="*.txt", loader_cls=TextLoader)
docs = loader.load()

## splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10, length_function=len)
chunked_docs = text_splitter.split_documents(docs)

doc_list = [doc.page_content for doc in chunked_docs]

## embedding the text
chroma_db = Chroma.from_documents(documents=chunked_docs, embedding=embeddings)

## retriever
retriever = chroma_db.as_retriever(search_kwagrs={'k': 4})

In [67]:
template = """
Answer the question only based on the context provided: {context}
Question: {question}
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

In [68]:
retrieval_chain = (
    RunnableParallel({'context': retriever, 'question': RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [69]:
retrieval_chain.invoke("what architecture is used in llama 3?")

'Llama 3 builds upon the standard decoder-only architecture.'

In [70]:
retrieval_chain.invoke("Highlight 2 key features of llama 3?")

'Llama 3 is the latest generation of open-source and its versatility enables a broad range.'

## Intro to tools and agents

In [71]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [72]:
print(tool.name)
print(tool.description)
print(tool.args)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}


In [73]:
print(tool.invoke("What is Llama 3?"))

Page: Llama (language model)
Summary: Llama ("Large Language Model Meta AI" serving as a backronym) is a family of large language models (LLMs) released by Meta AI starting in February 2023.
Llama models come in different sizes, ranging from 1 billion to 2 trillion parameters. Initially only a foundation model, starting with Llama 2, Meta AI released instruction fine-tuned versions alongside foundation models.
Model weights for the first version of Llama were only available to researchers on a case-by-case basis, under a non-commercial license. Unauthorized copies of the first model were shared via BitTorrent. Subsequent versions of Llama were made accessible outside academia and released under licenses that permitted some commercial use.
Alongside the release of Llama 3 and a standalone website, Meta added virtual assistant features to Facebook and WhatsApp in select regions; both services used a Llama 3 model. However, the latest version is Llama 4, released in April 2025.

Page: Lla

In [74]:
from langchain_community.tools import YouTubeSearchTool

yt_tool = YouTubeSearchTool()
print(yt_tool.name)
print(yt_tool.description)
print(yt_tool.args)

youtube_search
search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional
{'query': {'title': 'Query', 'type': 'string'}}


In [75]:
yt_tool.invoke("campusx")

"['https://www.youtube.com/watch?v=WzvURhaDZqI&pp=ygUHY2FtcHVzeA%3D%3D', 'https://www.youtube.com/watch?v=Zmy439spZB4&pp=ygUHY2FtcHVzeA%3D%3D']"

In [76]:
from langchain_tavily import TavilySearch

tavily_tool = TavilySearch()

In [77]:
tavily_tool.invoke({"query": "What happened at the last wimbledon"})

{'query': 'What happened at the last wimbledon',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.espn.com/tennis/story/_/id/45732583/jannik-sinner-defeats-carlos-alcaraz-rematch-win-wimbledon-2025-men-singles-title',
   'title': 'Jannik Sinner tops Carlos Alcaraz in rematch to win Wimbledon',
   'content': "Sinner closes '25 duel with Alcaraz, wins ATP title Top-ranked Jannik Sinner defeated two-time defending champion Carlos Alcaraz 4-6, 6-4, 6-4, 6-4 on Sunday to win his first Wimbledon championship and reverse the result of their epic French Open final last month. Exactly five weeks after the world No. 1 and No. 2 players completed the second-longest men's major final on the red clay of Roland Garros -- with Alcaraz overcoming a two-set deficit and saving three match points along the way -- it was Sinner who had to rally from dropping the opening set on grass to earn his fourth career Grand Slam title.",
   'score': 0.47314653,
   'raw

In [78]:
from langchain.agents import AgentType, load_tools, initialize_agent

tool = load_tools(["wikipedia"], llm=llm)
agent = initialize_agent(
    tools=tool,
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    
)
agent.invoke("What is India's GDP?")




> Entering new AgentExecutor chain...
Thought: I need to find out India's GDP. I will use Wikipedia for this.
Action:
```json
{
  "action": "wikipedia",
  "action_input": "India GDP"
}
```
Observation: Page: List of Indian states and union territories by GDP
Summary: These are lists of Indian states and union territories by their nominal gross state domestic product (GSDP). GSDP is the sum of all value added by industries within each state or union territory and serves as a counterpart to the national gross domestic product (GDP). As of 2011, the Government accounted for about 21% of the GDP followed by agriculture with 21% and corporate sector at 12%. The remaining 48% is sourced from small proprietorship and partnership companies, unorganized sector and households.



Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fourth-largest economy by nominal GDP and the third-largest by pu

{'input': "What is India's GDP?",
 'output': 'Agent stopped due to iteration limit or time limit.'}